<a href="https://colab.research.google.com/github/szymonbanach/Prolog-Zadania/blob/main/Zadania_1_5_Prolog.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Zadanie 1

Zadanie: Stwórz bazę faktów dla drzewa genealogicznego, która obejmuje relacje rodzic-dziecko, małżeństwo oraz płeć. Następnie zdefiniuj reguły przodek(X, Y), brat(X, Y), siostra(X, Y), kuzyn(X, Y).

In [ ]:
% Używam dyrektywy discontigous by fakty mogły być rozdzielone na grupy
:- discontiguous ojciec/2.
:- discontiguous matka/2.
:- discontiguous malzenstwo/2.
:- discontiguous mezczyzna/1.
:- discontiguous kobieta/1.

% Fakty o rodzicach1
ojciec(piotr, szymon).
ojciec(piotr, michal).
ojciec(piotr, izabela).
matka(agnieszka, szymon).
matka(agnieszka, michal).
matka(agnieszka, izabela).

% Fakty o relacji rodziców1
malzenstwo(piotr, agnieszka).

% Fakty o płci1
mezczyzna(piotr).
mezczyzna(szymon).
mezczyzna(michal).
kobieta(agnieszka).
kobieta(izabela).

%fakty rodzicach2
ojciec(lukasz, tomasz).
matka(ewa, tomasz).

%fakty o relacji rodziców2
malzenstwo(lukasz, ewa).

%fakty o płci2
mezczyzna(lukasz).
mezczyzna(tomasz).
kobieta(ewa).

%fakty o dziadkach
ojciec(jozef, agnieszka).
ojciec(jozef, ewa).
matka(krystyna, agnieszka).
matka(krystyna, ewa).
mezczyzna(jozef).
kobieta(krystyna).


% Reguła rodzica
rodzic(X, Y) :- ojciec(X, Y).
rodzic(X, Y) :- matka(X, Y).
% Przykładowe zapytanie: ?- rodzic(piotr, szymon).
% Oczekiwana odpowiedz: true


% Reguła przodka
przodek(X, Y) :- rodzic(X, Y).
% dodaję rekurencję do następnych pokoleń
przodek(X, Y) :-
  rodzic(X, Z),
  rodzic(Z, Y).
% Przykładowe zapytanie: ?- przodek(jozef, szymon).
% Oczekiwana odpowiedz: true

% Reguła malzenstwa
malzenstwo(X, Y) :-
    malzenstwo(Y, X).

% Reguła brata i siostry
brat(X, Y) :-
  rodzic(Z, X),
  rodzic(Z, Y),
  X \= Y, % X nie może być bratem samego siebie.
  mezczyzna(X).
% Przykładowe zapytanie: ?- brat(michal, szymon).
% Oczekiwana odpowiedz: true

siostra(X, Y) :-
  rodzic(Z, X),
  rodzic(Z, Y),
  X \= Y, % X nie może być siostrą samej siebie.
  kobieta(X).
% Przykładowe zapytanie: ?- siostra(agnieszka, ewa).
% Oczekiwana odpowiedz: true

% Reguła rodzeństwa potrzebna do reguły kuzyna
rodzenstwo(X, Y) :-
  brat(X, Y).
rodzenstwo(X, Y) :-
  siostra(X, Y).

% Reguła kuzyna
kuzyn(X, Y) :-
  rodzic(Z, X),
  rodzic(V, Y),
  rodzenstwo(Z, V). % rodzice kuzynów muszą być rodzeństwem
% Przykładowe zapytanie: ?- kuzyn(tomasz, szymon).
% Oczeikiwana odpowiedz: true

#Zadanie 2

Cel zadania: Stwórz i zaimplementuj bazę faktów dla systemu rezerwacji lotów, który umożliwia zarządzanie lotniskami, połączeniami między nimi oraz cenami biletów. Opracuj reguły, które pozwolą na zapytania o bezpośrednie połączenia, koszt podróży oraz znalezienie najkrótszej trasy między lotniskami.


In [ ]:
% Fakty o lotniskach
lotnisko(warszawa).
lotnisko(liverpool).
lotnisko(rzym).
lotnisko(oslo).
lotnisko(madryt).

% Fakty o połączeniach
polaczenie(warszawa, liverpool, 200).
polaczenie(liverpool, warszawa, 200).
polaczenie(warszawa, rzym, 300).
polaczenie(rzym, warszawa, 300).
polaczenie(warszawa, oslo, 600).
polaczenie(oslo, warszawa, 600).
polaczenie(liverpool, rzym, 650).
polaczenie(rzym, liverpool, 650).
polaczenie(liverpool, oslo, 850).
polaczenie(oslo, liverpool, 850).
polaczenie(liverpool, madryt, 490).
polaczenie(madryt, liverpool, 490).
polaczenie(rzym, madryt, 550).
polaczenie(madryt, rzym, 550).

% Reguła sprawdzająca istnienie bezpośredniego połączenia
polaczenie(X, Y) :-
    polaczenie(Y, X, _),
    write('Dostepne polaczenie bezposrednie').
% Przykładowe zapytanie: ?- polaczenie(warszawa, oslo).
% Oczekiwana odpowiedz: true

% Pomocnicza reguła wynajdująca trasę bezpośrednią
trasa(X, Y, _, C, [X, Y]) :-
    polaczenie(X, Y, C),
     X \= Y. % Miasto początkowe nie może być miastem docelowym
% Pomocnicza reguła wynajdujaca trasę pośrednią
trasa(X, Y, Odwiedzone, C, [X | ResztaS]) :- % Odwiedzone - lista punktów odwiedzonych już, [X | ResztaS(Reszta drogi)] - lista punktów jakie trzeba odwiedzić.
    polaczenie(X, Z, C1),
    lotnisko(X), lotnisko(Y), % Oba miasta muszą istnieć w bazie
    X \= Y, % Miasto początkowe nie może być miastem docelowym
    \+ member(Z, Odwiedzone), % Pilnuje czy punkt Z nie znajduje się w Odwiedzonych, by zapobiec zapętleniu
    trasa(Z, Y, [Z | Odwiedzone], C2, ResztaS), % Wykorzystanie rekurencji
    C is C1 + C2. % Pełna cena podróży składa się z cen połączeń bezpośrednich

% Reguła sprawdzająca koszt podróży.
koszt_podrozy(X, Y) :-
	lotnisko(X), lotnisko(Y),
    X \= Y,
    findall((C, S), trasa(X, Y, [X], C, S), Wyniki), % Funkcja findall zbiera wszystkie kombinacje cen i tras w liście wyniki. [X] - lista zawierająca punkt startowy, jest początkowym stanem listy Odwiedzone.
    findall(C, member((C, _), Wyniki), Lista), % Funkcja findall zbiera wszystkie ceny wywoływanego połączenia z listy Wyniki i umieszcza je w liście Lista.
	write('Ceny połączenia: '), write(Lista).
% Przykładowe zapytanie: ?- koszt_podrozy(rzym, madryt).
% Oczekiwana odpowiedz: Ceny połączenia: [550, 990, 2240, 1140]

% Moja reguła różni się od tej z polecenia, ponieważ pomyślałem, że dobrym pomysłem byłoby wypisanie wszystkich cen, niż żeby program wypisał pierwszą lepszą.
% W razie czego przedstawiam również niżej bardziej zbliżoną wersję do tej z polecenia:
koszt_podrozy1(X, Y, C) :-
    trasa(X, Y, [], C, _), !. % Korzystam z reguły trasa, w której są zawarte wszystkie potrzebne informacje, lecz czas będzie różnić się w zależności od kierunku przejazdu na tej samej trasie.
% Przykładowe zapytanie: ?- koszt_podrozy1(rzym, madryt, C).
% Oczekiwana odpowiedz: C = 550

% Regula wyznaczająca najkrótszą trasę wg. ceny
najkrotsza_droga(X, Y) :-
    findall((C, S), trasa(X, Y, [X], C, S), Wyniki),
    findall(C, member((C, _), Wyniki), Lista),
    min_list(Lista, MinC), % Wybiera najmniejsze wartości cen z listy.
    member((MinC, MinS), Wyniki), % MinC, MinS znajdują się w liście Wyniki.
    write('Koszt: '), write(MinC), nl, % nl dla nowej lini
    write('Trasa: '), write(MinS), !. % cut
% Przykładowe zapytanie: ?- najkrotsza_droga(oslo, madryt).
% Oczekiwana odpowiedz: Koszt: 1290, Trasa: [oslo, warszawa, liverpool, madryt]

Reguła koszt_podrozy posiada 2 wersje: koszt_podrozy(X, Y), oraz koszt_podrozy1(X, Y, C). Wersja koszt_podrozy(X, Y) zwraca wszystkie ceny danego połączenia.



#Zadanie 3

Cel zadania: Stwórz i zaimplementuj bazę faktów dla systemu zarządzania magazynem. Baza danych powinna zawierać informacje o produktach, ich ilościach oraz lokalizacji w magazynie. Opracuj reguły, które umożliwią zarządzanie dostępnością produktów, ich przenoszeniem oraz uzupełnianiem stanu.

In [ ]:
%Predykaty muszą być dynamiczne by zmieniać ich własności
:- dynamic produkt/3.

% Fakty o produktach
produkt(tv, 20, sektor_a).
produkt(smartfon, 54, sektor_a).
produkt(amplituner, 30, sektor_b).
produkt(gramofon, 15, sektor_b).
produkt(sluchawki, 40, sektor_c).
produkt(gitara, 12, sektor_c).

% Reguła dostępności produktu
dostepny_produkt(X) :-
    produkt(X, N, _),
    N > 0, % Jeśli ilość(N) jest większa niż 0, program zwróci wartość True
	write('Status produktu: Dostepny').
% Przykładowe zapytanie: ?- dostepny_produkt(tv).
% Oczekiwana odpowiedz: Status produktu: Dostepny

% Reguła przenosząca produkt do innego sektoru
przenies_produkt(X, Y) :-
    produkt(X, N, OldY),
    retract(produkt(X, N, OldY)), % usuwa
    assertz(produkt(X, N, Y)), % dodaje
	write(produkt(X, N, Y)). % Nowa lokalizacja produktu
% Przykładowe zapytanie: ?- przenies_produkt(tv, sektor_b).
% Oczekiwana odpowiedz: produkt(tv,20,sektor_b)

% Reguła dodającą ilość
uzupelnij_stan(X, N) :-
    produkt(X, N1, Y),
    N2 is N1 + N, % Nowa ilość to suma ilości początkowej i ilości podawanej(N)
    retract(produkt(X, N1, Y)), % usuwa
	assertz(produkt(X, N2, Y)), % dodaje
	write(produkt(X, N2, Y)). %Nowa ilość produktu
% Przykładowe zapytanie: ?- uzupelnij_stan(tv, 10).
% Oczekiwana odpowiedz: produkt(tv,30,sektor_b)

% P.S. Program nie działa w środowisku SWISH, dlatego trzeba go uruchamiać w SWI,
dlatego na GitHub załączam również plik z rozszerzeniem pl.

Dodatkowo dodałem write do reguł, by w odpowiedzi dostawać coś więcej niż true

#Zadanie 4

Cel zadania: Zaprojektuj i wdroż w Prologu bazę faktów dla systemu planowania trasy kuriera. Baza danych powinna zawierać informacje o miastach, połączeniach drogowych między nimi oraz czasie przejazdu. Opracuj reguły umożliwiające zapytania o dostępność dróg, obliczanie czasu przejazdu oraz wyznaczanie najkrótszej trasy.


In [ ]:
% Fakty dla miast
miasto(warszawa).
miasto(lublin).
miasto(krakow).
miasto(wroclaw).
miasto(gdansk).
miasto(poznan).

% Fakty dla połączeń bezpośrednich
droga(warszawa, lublin, 2).
droga(lublin, warszawa, 2).
droga(lublin, krakow, 3).
droga(krakow, lublin, 3).
droga(krakow, wroclaw, 3).
droga(wroclaw, krakow, 3).
droga(gdansk, poznan, 4).
droga(poznan, gdansk, 4).
droga(warszawa, krakow, 2).
droga(krakow, warszawa, 2).
droga(warszawa, gdansk, 3).
droga(gdansk, warszawa, 3).
droga(warszawa, poznan, 3).
droga(poznan, warszawa, 3).
droga(wroclaw, poznan, 1).
droga(poznan, wroclaw, 1).

% Reguły
droga(X, Y) :-
    droga(X, Y, _),
    write('Istnieje bezposrednie polaczenie: '), write(X), write(' - '), write(Y), !. % Dodałem to dodatkowo
% Przykładowe zapytanie: ?- droga(lublin, warszawa).
% Oczekiwana odpowiedz: Istnieje bezposrednie polaczenie: lublin - warszawa

% Pomocnicza reguła wynajdująca trasę bezpośrednią
trasa(X, Y, _, T, [X, Y]) :-
    droga(X, Y, T), % Bezpośrednie połączenie
    miasto(X), miasto(Y). % Oba miasta muszą istnieć w bazie
% Pomocnicza reguła wynajdujaca trasę pośrednią
trasa(X, Y, Odwiedzone, T, [X | ResztaS]) :- % Odwiedzone - lista punktów odwiedzonych już, [X | ResztaS(Reszta drogi)] - lista punktów jakie trzeba odwiedzić.
    droga(X, Z, T1),
    miasto(X), miasto(Y), % Oba miasta muszą istnieć w bazie
    X \= Y, % Miasto początkowe nie może być miastem docelowym
    \+ member(Z, Odwiedzone), % Pilnuje czy punkt Z nie znajduje się w Odwiedzonych, by zapobiec zapętleniu
    trasa(Z, Y, [Z | Odwiedzone], T2, ResztaS), % Wykorzystanie rekurencji
    T is T1 + T2. % Pełen czas podróży składa się z czasów indwywidualnych połączeń bezpośrednich

czas_przejazdu(X, Y) :-
    miasto(X), miasto(Y),
    X \= Y,
    findall((T, S), trasa(X, Y, [X], T, S), Wyniki), % Funkcja findall zbiera wszystkie kombinacje cen i tras w liście wyniki. [X] - lista zawierająca punkt startowy, jest początkowym stanem listy Odwiedzone.
    findall(T, member((T, _), Wyniki), Lista), % Funkcja findall zbiera wszystkie ceny wywoływanego połączenia z listy Wyniki i umieszcza je w liście Lista.
	write('Czasy przejazdu: '), write(Lista).
% Przykładowe zapytanie: ?- czas_przejazdu(gdansk, lublin).
% Oczekiwana odpowiedz: Czasy przejazdu: [9, 12, 11, 12, 5, 8, 13]

% Tak jak w zad.2 moja reguła różni się od tej z polecenia, ponieważ również pomyślałem, że dobrym pomysłem byłoby wypisanie wszystkich czasów, niż żeby program wypisał pierwszy lepszy.
% Wersja bardziej zbliżona do tej z polecenia poniżej:
czas_przejazdu1(X, Y, T) :-
    trasa(X, Y, [], T, _), !. % Korzystam z reguły trasa, w której są zawarte wszystkie potrzebne informacje, lecz czas będzie różnić się w zależności od kierunku przejazdu na tej samej trasie.
% Przykładowe zapytanie: ?- czas_przejazdu1(lublin, gdansk, T).
% Oczekiwana odpowiedz: Czasy przejazdu: T = 5

% Reguła wyznaczajaca najkrótszą trasę względem czasu przejazdu
najkrotsza_trasa(X, Y) :-
    findall((T, S), trasa(X, Y, [X], T, S), Wyniki), % Funkcja findall zbiera wszystkie kombinacje czasówn i tras w liście wyniki. [X] - lista zawierająca punkt startowy, jest początkowym stanem listy Odwiedzone.
    findall(T, member((T, _), Wyniki), Lista), % Funkcja findall zbiera wszystkie czasy wywoływanego połączenia z listy Wyniki i umieszcza je w liście Lista.
    min_list(Lista, MinT), % Wybiera najmniejsze wartości czsów z listy.
    member((MinT, MinS), Wyniki), % MinT, MinS znajdują się w liście Wyniki.
    write('Najkrotszy czas przejazdu: '), write(MinT), nl, % dla nowej linijki
    write('Trasa: '), write(MinS), !. % cut
% Przykładowe zapytanie: ?- najkrotsza_trasa(lublin, wroclaw).
% Oczekiwana odpowiedz: Najkrotszy czas przejazdu: 6, Trasa: [lublin, warszawa, poznan, wroclaw]

Reguła koszt_podrozy posiada 2 wersje: czas_przejazdu(X, Y), oraz czas_przejazdu1(X, Y, C). Wersja czas_przejazdu(X, Y) zwraca wszystkie czasy danego połączenia.

#Zadanie 5

Cel zadania: Stwórz system rekomendacji książek, który uwzględnia preferencje użytkowników, gatunki literackie oraz oceny książek. Twój system powinien być w stanie rekomendować książki danego gatunku użytkownikowi na podstawie jego preferencji oraz wcześniej przyznanych ocen.

In [ ]:
:- dynamic ocena/3.

% Fakty
ksiazka('Ogniem i mieczem', sienkiewicz, powiesc).
ksiazka('Mroki', gombrowicz, poezja).
ksiazka('Lalka', prus, powiesc).
ksiazka('Maly ksiaze', saint-exupery, powiesc).
ksiazka('Wybor poezji', lesmian, poezja).
ksiazka('Ojciec Chrzestny', puzo, powiesc).
ocena(uzytkownik1, 'Ogniem i mieczem', 2).
ocena(uzytkownik1, 'Ojciec Chrzestny', 5).
preferencje(uzytkownik1, powiesc).
preferencje(uzytkownik2, poezja).

% Dodanie nowej oceny
dodaj_ocene(X, Ksiazka, Ocena) :-
    assertz(ocena(X, Ksiazka, Ocena)), % Dodawanie nowych faktów (ocen).
	write('Pomyslnie dodano: '), write(ocena(X, Ksiazka, Ocena)).
% Przykładowe zapytanie: ?- dodaj_ocene(uzytkownik2, 'Wybor poezji', 5).
% Oczekiwana odpowiedz: Pomyslnie dodano: ocena(uzytkownik2,Wybor poezji,5)

% Usunięcie oceny
usun_ocene(X, Ksiazka) :-
    retract(ocena(X, Ksiazka, _)), % Usuwanie faktów (ocen).
    write('Pomyslnie usunieto ocene').
% Przykłaowe zapytanie: ?- usun_ocene(uzytkownik2, 'Wybor poezji').
% Oczekiwana odpowiedz: Pomyslnie usunieto ocene

% Reguła polecane_ksiazki/2
polecane_ksiazki(X, Gatunek) :-
    findall(
        Tytul, (ksiazka(Tytul, _, Gatunek), % Funkcja findall zbiera wszystkie tytuły ksiazek,
            preferencje(X, Gatunek),        % których gatunek pokrywa się z preferencjami uzytkownika,
            ocena(X, Tytul, Ocena),         % które zostały ocenione przez uzytkownika
            Ocena >= 3), 	                % na conajmniej 3
        PolecaneKsiazki), 	               % w liście PolecaneKsiazki
    write('Polecane ksiazki: '), write(PolecaneKsiazki).
% Przykładowe zapytanie: ?- polecane_ksiazki(uzytkownik1, powiesc).
% Oczekiwana odpowiedz: Polecane ksiazki: [Ojciec Chrzestny]

% P.S. Program nie działa w środowisku SWISH, dlatego trzeba go uruchamiać w SWI, dlatego na GitHub załączam również plik z rozszerzeniem pl


Tutaj również dodałem write, by urozmaicić odpowiedzi.